# Data Exploration

In [103]:
import pandas as pd

In [104]:
# Read in data
folder = 'tests/data'
commissions = pd.read_csv(f'{folder}/commissions.csv', parse_dates = ['date'], index_col=['date', 'vendor_id'])
order_lines = pd.read_csv(f'{folder}/order_lines.csv', index_col=['order_id', 'product_id'])
orders = pd.read_csv(f'{folder}/orders.csv',  parse_dates = ['created_at'], index_col='id')
product_promotions = pd.read_csv(f'{folder}/product_promotions.csv', parse_dates = ['date'], index_col=['date', 'product_id'])
products = pd.read_csv(f'{folder}/products.csv', index_col='id')
promotions = pd.read_csv(f'{folder}/promotions.csv', index_col='id')

In [105]:
order_lines.isnull().any()

product_description    False
product_price          False
product_vat_rate       False
discount_rate          False
quantity               False
full_price_amount      False
discounted_amount      False
vat_amount             False
total_amount           False
dtype: bool

In [106]:
test = order_lines[(order_lines.index.get_level_values('order_id') >= 2) & (order_lines.index.get_level_values('order_id') <= 10) ]
test['quantity'].sum()

30

In [107]:
orders.head()

,created_at,vendor_id,customer_id
id,,,
2,2020-04-01 18:43:57.052767,3,1398
3,2020-04-01 11:51:07.349383,2,7449


In [108]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 2 to 3
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   created_at   2 non-null      datetime64[ns]
 1   vendor_id    2 non-null      int64         
 2   customer_id  2 non-null      int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 64.0 bytes


In [109]:
products.head()

,description
id,
1,Oréos 2X
2,Škoda 3Tr
3,Oréos 4X
4,Škoda 6Tr
5,Škoda 7Tr


In [110]:
order_day = orders[orders['created_at'].dt.strftime('%Y-%m-%d') == '2019-08-02']
order_day

,created_at,vendor_id,customer_id
id,,,


In [111]:
test_order = order_lines.merge(orders, left_on='order_id', right_index=True)\
                .merge(products, left_on='product_id', right_index=True)
test_order.head()

product_description  product_price  product_vat_rate  \
order_id product_id                                                          
2        794                      IBM 032             10              0.11   
         780                      IBM 003             20              0.20   
         521         Deskjet 995c / 995ck             30              0.11   
3        372                       Kodiaq             10              0.11   
         316                    GLA-Class             20              0.11   

                     discount_rate  quantity  full_price_amount  \
order_id product_id                                               
2        794                   0.1        10                100   
         780                   0.2         2                 40   
         521                   0.3         2                 60   
3        372                   0.1        10                100   
         316                   0.2         2                 40   

                     discounted_amount  vat_amount  total_amount  \
order_id product_id                                                
2        794                        90        9.90         99.90   
         780                        32        6.40         38.40   
         521                        42        4.62         46.62   
3        372                        90        9.90         99.90   
         316                        32        3.52         35.52   

                                    created_at  vendor_id  customer_id  \
order_id product_id                                                      
2        794        2020-04-01 18:43:57.052767          3         1398   
         780        2020-04-01 18:43:57.052767          3         1398   
         521        2020-04-01 18:43:57.052767          3         1398   
3        372        2020-04-01 11:51:07.349383          2         7449   
         316        2020-04-01 11:51:07.349383          2         7449   

                              description  
order_id product_id                        
2        794                      IBM 032  
         780                      IBM 003  
         521         Deskjet 995c / 995ck  
3        372                       Kodiaq  
         316                    GLA-Class

In [112]:
# Order lines, orders and products
order_products = order_lines.merge(orders, left_on='order_id', right_index=True)
order_products = order_products.merge(products, left_on='product_id', right_index=True)
order_products.head()

product_description  product_price  product_vat_rate  \
order_id product_id                                                          
2        794                      IBM 032             10              0.11   
         780                      IBM 003             20              0.20   
         521         Deskjet 995c / 995ck             30              0.11   
3        372                       Kodiaq             10              0.11   
         316                    GLA-Class             20              0.11   

                     discount_rate  quantity  full_price_amount  \
order_id product_id                                               
2        794                   0.1        10                100   
         780                   0.2         2                 40   
         521                   0.3         2                 60   
3        372                   0.1        10                100   
         316                   0.2         2                 40   

                     discounted_amount  vat_amount  total_amount  \
order_id product_id                                                
2        794                        90        9.90         99.90   
         780                        32        6.40         38.40   
         521                        42        4.62         46.62   
3        372                        90        9.90         99.90   
         316                        32        3.52         35.52   

                                    created_at  vendor_id  customer_id  \
order_id product_id                                                      
2        794        2020-04-01 18:43:57.052767          3         1398   
         780        2020-04-01 18:43:57.052767          3         1398   
         521        2020-04-01 18:43:57.052767          3         1398   
3        372        2020-04-01 11:51:07.349383          2         7449   
         316        2020-04-01 11:51:07.349383          2         7449   

                              description  
order_id product_id                        
2        794                      IBM 032  
         780                      IBM 003  
         521         Deskjet 995c / 995ck  
3        372                       Kodiaq  
         316                    GLA-Class

In [113]:
# 1. Total number of items sold on a day
date_str = '2020-04-01'
day_orders = order_products[order_products['created_at'].dt.strftime('%Y-%m-%d') == date_str].sort_values(by=['order_id', 'product_id'])
num_items_sold = day_orders['quantity'].sum()
num_items_sold

30

In [114]:
# 1. Total number of items sold on a day
date_str = '2020-04-01'
day_orders = order_products[order_products['created_at'].dt.strftime('%Y-%m-%d') == date_str].sort_values(by=['order_id', 'product_id'])
num_items_sold = day_orders['quantity'].sum()
num_items_sold

30

In [115]:
# 2. Total number of customers
unique_customers = pd.unique(day_orders['customer_id'])
unique_customers, len(unique_customers)

(array([1398, 7449]), 2)

In [116]:
# 3. Total maount of discount given on day
day_orders['discount_value'] = day_orders['full_price_amount'] - day_orders['discounted_amount']
total_discount = day_orders['discount_value'].sum()
total_discount

72

In [117]:
# 4. Average discount rate applied to items sold
average_discount = day_orders.loc[day_orders['discount_rate'] > 0, 'discount_rate'].mean()
average_discount

0.19999999999999998

In [120]:
day_orders

product_description  product_price  \
order_id product_id                                                      
2        521                       Deskjet 995c / 995ck             30   
         780                                    IBM 003             20   
         794                                    IBM 032             10   
3        266         Wrangler Unlimited/Wrangler 4 door             40   
         316                                  GLA-Class             20   
         372                                     Kodiaq             10   
         843                                 A910/A910i             30   

                     product_vat_rate  discount_rate  quantity  \
order_id product_id                                              
2        521                     0.11            0.3         2   
         780                     0.20            0.2         2   
         794                     0.11            0.1        10   
3        266                     0.00            0.0         2   
         316                     0.11            0.2         2   
         372                     0.11            0.1        10   
         843                     0.11            0.3         2   

                     full_price_amount  discounted_amount  vat_amount  \
order_id product_id                                                     
2        521                        60                 42        4.62   
         780                        40                 32        6.40   
         794                       100                 90        9.90   
3        266                        80                 80        0.00   
         316                        40                 32        3.52   
         372                       100                 90        9.90   
         843                        60                 42        4.62   

                     total_amount                 created_at  vendor_id  \
order_id product_id                                                       
2        521                46.62 2020-04-01 18:43:57.052767          3   
         780                38.40 2020-04-01 18:43:57.052767          3   
         794                99.90 2020-04-01 18:43:57.052767          3   
3        266                80.00 2020-04-01 11:51:07.349383          2   
         316                35.52 2020-04-01 11:51:07.349383          2   
         372                99.90 2020-04-01 11:51:07.349383          2   
         843                46.62 2020-04-01 11:51:07.349383          2   

                     customer_id                         description  \
order_id product_id                                                    
2        521                1398                Deskjet 995c / 995ck   
         780                1398                             IBM 003   
         794                1398                             IBM 032   
3        266                7449  Wrangler Unlimited/Wrangler 4 door   
         316                7449                           GLA-Class   
         372                7449                              Kodiaq   
         843                7449                          A910/A910i   

                     discount_value  
order_id product_id                  
2        521                     18  
         780                      8  
         794                     10  
3        266                      0  
         316                      8  
         372                     10  
         843                     18

In [128]:
day_orders.index.get_level_values(0).unique(), day_orders.index

(Int64Index([2, 3], dtype='int64', name='order_id'),
 MultiIndex([(2, 521),
             (2, 780),
             (2, 794),
             (3, 266),
             (3, 316),
             (3, 372),
             (3, 843)],
            names=['order_id', 'product_id']))

In [130]:
# 5. Average order total for that day
total_day_revenue = day_orders['total_amount'].sum()
num_orders_for_day = len(day_orders.index.get_level_values(0).unique())
average_day_revenue = total_day_revenue / num_orders_for_day
print(total_day_revenue), print(num_orders_for_day), print(total_day_revenue)

446.96000000000004
2
446.96000000000004


(None, None, None)

In [87]:
day_prod_promos = product_promotions[product_promotions.index.get_level_values('date') == date_str].reset_index()
day_prod_promos

,date,product_id,promotion_id
0,2020-04-01,794,2
1,2020-04-01,780,2
2,2020-04-01,316,3
3,2020-04-01,843,3


In [88]:
commissions.head()

rate
date       vendor_id      
2020-04-01 1          0.29
           2          0.07
           3          0.27
           4          0.14
           5          0.05

In [89]:
day_commissions = commissions[commissions.index.get_level_values('date') == date_str].reset_index()
day_commissions

,date,vendor_id,rate
0,2020-04-01,1,0.29
1,2020-04-01,2,0.07
2,2020-04-01,3,0.27
3,2020-04-01,4,0.14
4,2020-04-01,5,0.05
5,2020-04-01,6,0.24
6,2020-04-01,7,0.29
7,2020-04-01,8,0.22
8,2020-04-01,9,0.24


In [90]:
day_orders.head()

product_description  product_price  \
order_id product_id                                                      
2        521                       Deskjet 995c / 995ck             30   
         780                                    IBM 003             20   
         794                                    IBM 032             10   
3        266         Wrangler Unlimited/Wrangler 4 door             40   
         316                                  GLA-Class             20   

                     product_vat_rate  discount_rate  quantity  \
order_id product_id                                              
2        521                     0.11            0.3         2   
         780                     0.20            0.2         2   
         794                     0.11            0.1        10   
3        266                     0.00            0.0         2   
         316                     0.11            0.2         2   

                     full_price_amount  discounted_amount  vat_amount  \
order_id product_id                                                     
2        521                        60                 42        4.62   
         780                        40                 32        6.40   
         794                       100                 90        9.90   
3        266                        80                 80        0.00   
         316                        40                 32        3.52   

                     total_amount                 created_at  vendor_id  \
order_id product_id                                                       
2        521                46.62 2020-04-01 18:43:57.052767          3   
         780                38.40 2020-04-01 18:43:57.052767          3   
         794                99.90 2020-04-01 18:43:57.052767          3   
3        266                80.00 2020-04-01 11:51:07.349383          2   
         316                35.52 2020-04-01 11:51:07.349383          2   

                     customer_id                         description  \
order_id product_id                                                    
2        521                1398                Deskjet 995c / 995ck   
         780                1398                             IBM 003   
         794                1398                             IBM 032   
3        266                7449  Wrangler Unlimited/Wrangler 4 door   
         316                7449                           GLA-Class   

                     discount_value  
order_id product_id                  
2        521                     18  
         780                      8  
         794                     10  
3        266                      0  
         316                      8

In [102]:
# 6. Total amount of commissions for that day
day_orders_commissions = day_orders.reset_index()
# day_orders_commissions = day_orders.merge(day_prod_promos, how='left', left_on='product_id', right_on='product_id', suffixes=('_orders','_promos'))
day_orders_commissions = day_orders_commissions.merge(day_commissions, how='left', left_on='vendor_id', right_on='vendor_id', suffixes=('','_commissions'))
day_orders_commissions['commission_amount'] = day_orders_commissions['total_amount'] * day_orders_commissions['rate']
day_orders_commissions

,order_id,product_id,product_description,product_price,product_vat_rate,discount_rate,quantity,full_price_amount,discounted_amount,vat_amount,total_amount,created_at,vendor_id,customer_id,description,discount_value,date,rate,commission_amount
0,2,521,Deskjet 995c / 995ck,30,0.11,0.3,2,60,42,4.62,46.62,2020-04-01 18:43:57.052767,3,1398,Deskjet 995c / 995ck,18,2020-04-01,0.27,12.5874
1,2,780,IBM 003,20,0.20,0.2,2,40,32,6.40,38.40,2020-04-01 18:43:57.052767,3,1398,IBM 003,8,2020-04-01,0.27,10.3680
2,2,794,IBM 032,10,0.11,0.1,10,100,90,9.90,99.90,2020-04-01 18:43:57.052767,3,1398,IBM 032,10,2020-04-01,0.27,26.9730
3,3,266,Wrangler Unlimited/Wrangler 4 door,40,0.00,0.0,2,80,80,0.00,80.00,2020-04-01 11:51:07.349383,2,7449,Wrangler Unlimited/Wrangler 4 door,0,2020-04-01,0.07,5.6000
4,3,316,GLA-Class,20,0.11,0.2,2,40,32,3.52,35.52,2020-04-01 11:51:07.349383,2,7449,GLA-Class,8,2020-04-01,0.07,2.4864
5,3,372,Kodiaq,10,0.11,0.1,10,100,90,9.90,99.90,2020-04-01 11:51:07.349383,2,7449,Kodiaq,10,2020-04-01,0.07,6.9930
6,3,843,A910/A910i,30,0.11,0.3,2,60,42,4.62,46.62,2020-04-01 11:51:07.349383,2,7449,A910/A910i,18,2020-04-01,0.07,3.2634


In [92]:
total_commissions_for_day = day_orders_commissions['commission_amount'].sum()
total_commissions_for_day

68.27120000000001

In [93]:
day_orders_commissions.loc[day_orders_commissions['order_id'] == 97, 'commission_amount'].mean()

nan

In [100]:
day_orders_commissions[['order_id', 'commission_amount']].groupby('order_id').sum().mean()

commission_amount    34.1356
dtype: float64

In [101]:
# 7. Average amount of commission per order
average_commissions = day_orders_commissions[['order_id','commission_amount']].groupby('order_id').sum().mean()
average_commissions[0], type(average_commissions[0])

(34.135600000000004, numpy.float64)

In [96]:
# 8. Total amount of commissions earned per promotion
commissions_per_promo = day_orders_commissions.merge(day_prod_promos, how='inner', left_on='product_id', right_on='product_id', suffixes=('','_promos'))
commissions_per_promo[['commission_amount', 'promotion_id', ]].groupby('promotion_id').sum()

,commission_amount
promotion_id,
2,37.3410
3,5.7498


In [97]:
commissions_check = day_orders_commissions.loc[day_orders_commissions['product_id'] == 272, 'commission_amount'].sum()
commissions_check

0.0

In [98]:
prods = pd.unique(day_orders_commissions['product_id']).tolist()
len(prods)
check_prods = day_prod_promos['product_id'].tolist()
check_prods
found = [cp for cp in check_prods if cp in prods]
found

[794, 780, 316, 843]